In [1]:
###

# Адаменко Павел
## Дз: HITS + PageRank.

In [94]:
#imports
import time
import sys
import zlib
import base64
import re
from bs4 import BeautifulSoup
import sys
import glob
import sys

Директории и файлы для загрузки и выгрузки файлов

In [107]:
#Files innput:
FILEURL   = 'lenta/1_10/urls.txt' 
DOCSFILES = 'lenta/1_10/docs*'

#Files output:
OUTPUTGRAPHFILE = 'dataNew/1_10/graph1_10.txt'

OUTPUTPAGERANK  = 'dataNew/1_10/PageRank1_10.txt'
OUTPUTHITS      = 'dataNew/1_10/HITS1_10.txt'

Словари соответствия url:id и id:url 

(Вопрос: существует ли в питоне биективный словарь?)

In [108]:
#Build url_id, id_url dicts
url_id = {}
id_url = {}

def parse_single_url(url):
    return url[url.find("://lenta.ru")+len("://lenta.ru"):]

with open(FILEURL, 'r') as input_file:
    for line in input_file:
        line_parts = line.rstrip().split('\t')
        doc_id = int(line_parts[0])
        url = line_parts[1]
        pure_url = parse_single_url(url)
        url_id[doc_id] = pure_url
        id_url[pure_url] = doc_id

# Класс построения графа:
записывает файл OUTPUTGRAPHFILE в формате:

id1 : id2,id3,..idn

id2 : ...


Где id2, id3,.. idn это ссылки на известные url_id с id1

In [6]:
class make_graph:
    
    input_path  = ''
    output_file = ''
    
    def __init__(self, input_path, output_file):
        self.input_path  = input_path
        self.output_file = output_file
    
    def get_urls_from_html(self, html, output_file):
        links = BeautifulSoup(html, 'html').findAll('a')
        url_list = []
        for link in links:
            url = link.get('href')
            if url in id_url:
                if id_url[url] not in url_list:
                    url_list.append(id_url[url])
        for c_id, doc_id in enumerate(sorted(url_list)):
            if c_id == 0:
                output_file.write('%d' % doc_id)
            else: 
                output_file.write(',%d' % doc_id)
        return 

    def build_graph(self):
        output_file = self.output_file
        input_path  = self.input_path
        with open(output_file, 'w') as output_file:
            len_file_count = len(glob.glob(input_path))
            for file_count, file_name in enumerate(glob.glob(input_path)):
                with open(file_name, 'r') as input_file:
                    TEXT = input_file.read()
                    LEN = TEXT.count('\n')-1
                    for line_id, line in enumerate(TEXT.split('\n')):
                        if '\t' not in line: continue
                        line_parts = line.split('\t')
                        doc_id = int(line_parts[0])
                        based64 = line_parts[1] 
                        sys.stdout.write('\r: %d / %d doc_id: %d (files %d / %d)' % (line_id, LEN, doc_id, file_count+1, len_file_count))

                        html = base64.decodestring(based64).decode('zlib')
                        output_file.write('%d : ' % doc_id)
                        self.get_urls_from_html(html, output_file)
                        output_file.write('\n')

        return 

Самый долгий кусок кода около 10 часов, на всей выборке

In [7]:
#speed ~ 1.2 url/sec
# 32000*1.2 /60/60 ~ 11 hours
start_time = time.time()
worker = make_graph(DOCSFILES, OUTPUTGRAPHFILE)

####work for 8 hours####
worker.build_graph() #
########################

print("\n--- %s seconds ---" % (time.time() - start_time))


: 0 / 564 doc_id: 0 (files 1 / 1)

/Users/padam/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


: 564 / 564 doc_id: 564 (files 1 / 1)
--- 792.896014929 seconds ---


# Класс для работы с полученным графом
(описание в коментах)

In [109]:
import sys
import math
import random

class load_graph:
    # загружаем граф из файла, превращаем его в словарь вида:
    # {id: {'to':[id1,id2,..],'from':[id1,id2,..]}..}
    def __init__(self, input_file_name = OUTPUTGRAPHFILE):
        try:
            graph = {}
            with open(input_file_name, 'r') as input_file:

                for line in input_file:

                    line_parts = line.rstrip().split(' : ')
                    if len(line_parts) < 2: 
                        continue

                    doc_id = int(line_parts[0])
                    links_to =  [
                        int(linked_id) 
                        for linked_id in line_parts[1].split(',')#[:-1]
                    ]

                    if doc_id in graph:
                        graph[doc_id]['to'].extend(links_to)
                    else:
                        graph[doc_id] = {
                            'to' : links_to,
                            'from' : []
                        }

                    for linked_id in links_to:
                        if linked_id in graph:
                            graph[linked_id]['from'].append(doc_id)
                        else:
                            graph[linked_id] = {
                                'to' : [],
                                'from' : [doc_id]
                            }
                    self.graph = graph
                    
            sys.stdout.write(" SUCCESS: load graph file: "+str(input_file_name)+'\n')
        except:
            sys.stderr.write(" FAIL:    Something wrong with load graph: "+str(input_file_name)+'\n')
    
#     def get_graph(self):
#         return self.graph
    
    def _iterationHITS(self, graph):
            for doc_id in graph:
                auth_to = [ graph[to_id]['author_old']  for to_id in graph[doc_id]['to'] ]
                
                graph[doc_id]['hubs_new'] = sum(auth_to)

                hubs_from = [ graph[from_id]['hubs_old'] for from_id in graph[doc_id]['from']]
                graph[doc_id]['author_new'] = sum(hubs_from)
            max_auth = math.sqrt(sum([graph[doc_id]['author_new']**2 for doc_id in graph]))
            max_hubs = math.sqrt(sum([graph[doc_id]['hubs_new']**2   for doc_id in graph]))
            for doc_id in graph:
                graph[doc_id]['author_new'] /= 1.0*max_auth
                graph[doc_id]['hubs_new']   /= 1.0*max_hubs
                graph[doc_id]['author_old']  = graph[doc_id]['author_new']
                graph[doc_id]['hubs_old']    = graph[doc_id]['hubs_new']

            return graph
        
    def HITS(self, n_iterations=10):
        # algorithm from here https://en.wikipedia.org/wiki/HITS_algorithm
        try:
            graph = self.graph
            self.HITiler  = n_iterations
            for doc_id in graph:
                graph[doc_id]['author_old'] = 1
                graph[doc_id]['author_new'] = 1
                graph[doc_id]['hubs_old']   = 1
                graph[doc_id]['hubs_new']   = 1

            for i in xrange(n_iterations):
                sys.stdout.write('\r: '+str(i+1)+'/'+str(n_iterations)+'...')
                graph = self._iterationHITS(graph)
            sys.stdout.write('\r SUCCESS: Calculate HITs ( iter='+str(n_iterations)+')\n')
            self.HITS = graph
            return True   
        except:
            sys.stderr.write('\r FAIL:    Something wrong with HITs calculation\n')
            return False
    
    def _iterationPR(self, graph, alpha, N):
        for doc_id in graph:
            PR_from_sum = sum([(graph[id_from]['old']*1.0 / len(graph[id_from]['to'])) 
                               for id_from in graph[doc_id]['from']
                                ]
                            )

                        
            graph[doc_id]['new'] = alpha*(1.0/N) + (1-alpha)*PR_from_sum
            
        # dangling nodes teleport 
        for doc_id in graph:
            if graph[doc_id]['from'] == []:
                    goList = []
                    for go_id in graph:
                        if random.random()>alpha:
                            goList.append(go_id)
                    for go_id in goList:
                        graph[go_id]['new'] += (1-alpha)*graph[doc_id]['old']*1.0/len(goList)
                        
        for doc_id in graph:
            graph[doc_id]['old'] = graph[doc_id]['new']
#             graph[doc_id]['new'] = 0
        
        return graph
    
    def PageRank(self, alpha=0.85, n_iterations=10):
        # algorithm from here https://en.wikipedia.org/wiki/PageRank
        # dangling nodes treated by random teleport 
        try:
            self.PRalpha = alpha
            self.PRiter  = n_iterations
            graph = self.graph
            N = len(graph)
            for doc_id in graph:
                graph[doc_id]['old'] = 1.0/N
                graph[doc_id]['new'] = 1.0/N  

            for i in xrange(n_iterations):
                sys.stdout.write('\r: '+str(i+1)+'/'+str(n_iterations)+'...')
                graph = self._iterationPR(graph, alpha, N)
            sys.stdout.write('\r SUCCESS: Calculate Page Rank ( alpha='+str(alpha)+', iter='+str(n_iterations)+' )\n')
            self.PR = graph
            return True 
        except:
            sys.stderr.write('\r FAIL:    Something wrong with PageRank calculation\n')
            return False
    
    def print_best_by_HITS(self, url_id = url_id, output_file_name=OUTPUTHITS, n_best=30):
        try:
            graph = self.HITS
            
            sorted_by_auth = sorted(graph.keys(), key=lambda v: float(graph[v]['author_new']), reverse=True)
            sorted_by_hubs = sorted(graph.keys(), key=lambda v: float(graph[v]['hubs_new']),   reverse=True)

            with open(output_file_name, 'w') as output_file:

                output_file.write('\tBest HITs authorities ( iter = '+str(self.HITiler)+' ) :\n\n')
                for doc_id in sorted_by_auth[:n_best]:
                    output_file.write(str(doc_id) + '\thttp://lenta.ru' +str(url_id[doc_id])+'\t'+str(graph[doc_id]['author_new'])+'\n')



                output_file.write('\n')
                output_file.write('===================================') 
                output_file.write('\n\n\tBest HITs hubs:\n\n')
                
                for doc_id in sorted_by_hubs[:n_best]:
                    output_file.write(str(doc_id) + '\thttp://lenta.ru' +str(url_id[doc_id])+'\t'+str(graph[doc_id]['hubs_new'])+'\n')
                    
            sys.stdout.write(" SUCCESS: Output for HITS in "+str(output_file_name)+'\n')
            return True
        except:
            sys.stderr.write(" FAIL:    Something wrong with HITS output :(\n")
            return False
        
    def print_best_by_PageRank(self, url_id = url_id, output_file_name=OUTPUTPAGERANK, n_best=30):
        try:
            graph = self.PR
            sorted_by_PR = sorted(graph.keys(), key=lambda v: float(graph[v]['new']), reverse=True)
            
            with open(output_file_name, 'w') as output_file:
                output_file.write('\tBest PageRank authorities ( iter='+str(self.PRiter)+', alpha='+str(self.PRalpha)+' ) :\n\n')
                for doc_id in sorted_by_PR[:n_best]:
                    output_file.write(str(doc_id) + '\thttp://lenta.ru' +str(url_id[doc_id])+'\t'+str(graph[doc_id]['new'])+'\n')
                    
            sys.stdout.write(" SUCCESS: Output for PageRank in "+str(output_file_name)+'\n')
            return True
        except:
            sys.stderr.write(" FAIL:    Something wrong with PageRank output :(\n")
            return False
            
        

In [ ]:
if __name__ == "__main__":
    worker1 = load_graph()
#     print(worker1.graph)
#     graph = worker.get_graph()
    HITS = worker1.HITS()
    PR   = worker1.PageRank()
    worker1.print_best_by_HITS()
    worker1.print_best_by_PageRank()


 SUCCESS: load graph file: dataNew/1_10/graph1_10.txt
 SUCCESS: Calculate HITs ( iter=10)
: 1/10...